In [16]:
# ! pip install selenium

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
from datetime import datetime
from selenium import webdriver
import time

In [2]:
# url을 담은 txt 파일 읽어오기 
with open('C:/Users/이찬희/python2021/url.txt', 'rt', encoding='UTF8') as file:
    content = list()

    while True:
        sentence = file.readline()

        if sentence:
            content.append(sentence)
        else:
            break
            
    print(content)

['https://kr.iherb.com/pr/California-Gold-Nutrition-Omega-800-Pharmaceutical-Grade-Fish-Oil-80-EPA-DHA-Triglyceride-Form-1000-mg-90-Fish-Gelatin-Softgels/85180\n', 'https://kr.iherb.com/pr/California-Gold-Nutrition-Omega-800-Pharmaceutical-Grade-Fish-Oil-80-EPA-DHA-1-000-mg-30-Fish-Gelatin-Softgels/82845\n', 'https://kr.iherb.com/pr/California-Gold-Nutrition-LactoBif-Probiotics-30-Billion-CFU-60-Veggie-Capsules/64009?rec=iherbtest-home\n', 'https://kr.iherb.com/pr/PEAKfresh-USA-Produce-Bags-with-Twist-Ties-Reusable-10-Bags/44160\n', 'https://kr.iherb.com/pr/California-Gold-Nutrition-Gold-C-Powder-Vitamin-C-1-000-mg-8-81-oz-250-g/59561\n', 'https://kr.iherb.com/pr/Kosette-Sanitizer-Wipes-Alcohol-Free-20-Wipes/103742']


In [3]:
# 브라우저 없이 selenium 사용 
webdriver_options = webdriver.ChromeOptions()
webdriver_options .add_argument('headless')

# selenium에서 사용할 웹 드라이버 절대 경로 정보
chromedriver = 'C:/Users/이찬희/python2021/chromedriver.exe'

In [4]:
# selenum의 webdriver에 앞서 설치한 chromedirver를 연동한다.
driver = list()
for i in (content):
    i = webdriver.Chrome(chromedriver, options = webdriver_options )
    driver.append(i)

In [5]:
# driver로 특정 페이지를 연다.
for i in  range (len(content)):
    driver[i].get(content[i])

In [6]:
# 해당 페이지의 상태를 가져온다. 
locate= '#stock-status > div.text-danger.stock-status-text'

# 상품의 고유번호을 가져온다. 
n_locate = '#product-specs-list > li:nth-child(4) > span'

# 상품의 full name
f_locate = '#name'

In [7]:
name_list = list()
full_list = list()
for i in driver:
    name_list.append(i.find_element_by_css_selector(n_locate).text)
    full_list.append(i.find_element_by_css_selector(f_locate).text)

print(name_list)
print(full_list)

['898220012664', '898220012510', 'CGN-00965', '892453002003', 'CGN-00935', 'IMI-21029']
['California Gold Nutrition, Omega 800 제약 등급 피쉬 오일, EPA/DHA 80%, 트라이글리세라이드 형태, 1,000mg, 피쉬 젤라틴 소프트젤 90정', 'California Gold Nutrition, Omega 800 제약 등급 피쉬 오일, EPA/DHA 80%, 1,000mg, 피쉬 젤라틴 소프트젤 30정', 'California Gold Nutrition, LactoBif 프로바이오틱, 300억 CFU, 베지 캡슐 60정', 'PEAKfresh USA, 신선식품 보관백(묶는 끈 포함), 재사용 가능, 10개', 'California Gold Nutrition, Gold C 파우더, 비타민C, 1,000mg, 250g(8.81oz)', 'Kosette, 손소독 티슈, 알코올 무함유, 20개입']


In [8]:
import openpyxl
from openpyxl.styles import PatternFill, Color

# 워크북(엑셀파일)을 새로 만듭니다.
wb = openpyxl.Workbook()

# 1번 시트 설정
sheet1 = wb['Sheet']
sheet1.title = name_list[0] # 시트 name을 고유 번호로 설정하기 
sheet1.append(["날짜", "시간", "재고여부"]) # 헤더 추가하기
sheet1.cell(row=1, column=4).value = full_list[0] # (D,1)에 이름 추가  
sheet1.column_dimensions['D'].width = 90
sheet1.column_dimensions['A'].width = 15
sheet1.cell(row=1, column=5).value = content[0] #(F,1)에 해당 상품 url 추가 

# 1번 제외 나머지 시트 설정 
for i in range (1, len(name_list)):
    sheeti = wb.create_sheet(name_list[i])

    sheeti.append(["날짜", "시간", "재고여부"])
    sheeti.cell(row=1, column=4).value = full_list[i]
    sheeti.column_dimensions['D'].width = 90
    sheeti.column_dimensions['A'].width = 15
    sheeti.cell(row=1, column=5).value = content[i]

wb.save('재고확인.xlsx')

In [9]:
wb = openpyxl.load_workbook('재고확인.xlsx')
status_list = list()
num = 0
count = 0

while True:  
    try: 
        now = datetime.now()
        date = "%s년 %s월 %s일" %(now.year, now.month, now.day)
        hour = "%s시 %s분" %(now.hour, now.minute)

        #상품의 재고 상황을 가져온다. 
        for i in driver:
            try:
                status_list.append(i.find_element_by_css_selector(locate).text)

            except:
                status_list.append('재고 있음')

        #각 시트에 시간과 상태를 저장한다.
        for i in range (len (name_list)):
            if (status_list[num] == '품절'):
                sheeti = wb[name_list[i]]
                sheeti.append([date, hour, status_list[num]])
                sheeti.cell(row = count+2, column = 3).fill = PatternFill(start_color='00FF0000', end_color='00FF0000', fill_type='solid')
                num += 1 # 새로 저장한 상태로 업데이트 

            elif (status_list[num] == '재고 있음'):
                sheeti = wb[name_list[i]]
                sheeti.append([date, hour, status_list[num]])
                sheeti.cell(row = count+2, column = 3).fill = PatternFill(start_color='0099CC00', end_color='0099CC00', fill_type='solid')
                num += 1 # 새로 저장한 상태로 업데이트 
        
        #1분마다 정보를 갱신한다. 
        time.sleep(59) 

        #파일에 저장한다. 
        wb.save('재고확인.xlsx') 
        count += 1 # 각 시트 셀의 num
              
    except:
        print("Error")
        break

Error
